### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [1]:
#installation step
!pip install transformers
!pip install sentencepiece
!pip install Path
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train
!mkdir data/test
!mkdir data/val
!mkdir data/vocab
#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

     |████████████████████████████████| 573kB 8.8MB/s 
     |████████████████████████████████| 1.0MB 59.2MB/s 
     |████████████████████████████████| 3.7MB 57.2MB/s 
     |████████████████████████████████| 890kB 79.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=c83f452ddc01ee3efa09908f4bf36c2cd7dcdc7cd43f10468d32b1fda519c6ff
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
--2020-04-22 14:11:56--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 173.194.79.139, 173.194.79.101, 173.194.79.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.79.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hbgbti4o25bpcl93udfvqqlhondek

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchtext.datasets.translation import TranslationDataset
from torchtext.data import Field, BucketIterator

import sentencepiece as spm

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, T5Model


from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab

from sklearn.model_selection import train_test_split


In [3]:
ls data/AD_NMT-master

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [5]:
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])
print(len(data_English_MSA_trainval))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [6]:
ls data/AD_NMT-master/

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
eng_msa_train, eng_msa_val = train_test_split(data_English_MSA_trainval, test_size=.2)
eng_msa_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2)
mag_msa_test = data_Magribi_MSA_test

In [8]:
print(len(eng_msa_train))
print(len(eng_msa_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [0]:
file_path = 'data/'

def pytorch_format(ds, src='en', trg='msa', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + src
    trg_formatted = datatype + '_' + src + '_' + trg + '.' + trg
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as srctxt, open(file_path + datatype + "/" + trg_formatted, 'wt') as trgtxt:
        for i, arr in enumerate(ds):
            srctxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
            trgtxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')

In [0]:
#splits each language pair file into datasets of single language, to be merged again by the pytorch dataset class later

pytorch_format(eng_msa_train, 'eng', 'msa', 'train')
pytorch_format(eng_msa_val, 'eng', 'msa', 'val')
pytorch_format(eng_msa_test, 'eng', 'msa', 'test')

pytorch_format(lav_msa_train, 'lav', 'msa', 'train')
pytorch_format(lav_msa_val, 'lav', 'msa', 'val')
pytorch_format(lav_msa_test, 'lav', 'msa', 'test')

pytorch_format(mag_msa_train, 'mag', 'msa', 'train')
pytorch_format(mag_msa_val, 'mag', 'msa', 'val')
pytorch_format(mag_msa_test, 'mag', 'msa', 'test')

In [0]:
cat data/train/train_eng_msa.eng

In [0]:
cat data/train/train_eng_msa.msa

In [0]:
cat data/train/train_lav_msa.lav

In [0]:
cat data/train/train_lav_msa.msa

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [15]:
!mkdir data/vocab
!mkdir data/model

mkdir: cannot create directory ‘data/vocab’: File exists


In [0]:
"""
Testing:
Create a text file with all the vocab available from all sources for each language for SentencePiece to create a library 

TODO: implement bpemb model for MSA vocab/train SPM on larger datasets for dialects
"""

en_vocab = open("data/vocab/eng_vocab.txt", "wt")
msa_vocab = open("data/vocab/msa_vocab.txt", "wt")
lav_vocab = open("data/vocab/lav_vocab.txt", "wt")
mag_vocab = open("data/vocab/mag_vocab.txt", "wt")

MSA_text = ""
EN_text = ""

def create_vocab(file='', src='en_vocab', tgt='msa_vocab'):
  for line in file:
        src_sent = line[0]
        src_words = src_sent.split(" ")
        for count, word in enumerate(src_words):
            src.write(word)
        src.write("\n")
        
        tgt_sent = line[1]
        tgt_words = tgt_sent.split(" ")
        for count, word in enumerate(tgt_words):
            tgt.write(word)
        tgt.write("\n")

create_vocab(data_English_MSA_both, en_vocab, msa_vocab)
create_vocab(data_LAV_MSA_both, lav_vocab, msa_vocab)
create_vocab(data_Magribi_MSA_both, mag_vocab, msa_vocab)

en_vocab.close()
msa_vocab.close()
lav_vocab.close()
mag_vocab.close()

In [17]:
spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt --model_prefix=data/model/eng --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/vocab/msa_vocab.txt --model_prefix=data/model/msa --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/vocab/lav_vocab.txt --model_prefix=data/model/lav --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/vocab/mag_vocab.txt --model_prefix=data/model/mag --vocab_size=2000')
#

True

In [18]:
sp = spm.SentencePieceProcessor()
sp.load('data/eng.model')

True

In [19]:
ls data

AD_NMT-master/  eng.vocab  lav.vocab  mag.vocab  msa.vocab  train/  vocab/
eng.model       lav.model  mag.model  msa.model  test/      val/


In [20]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[89, 83, 12, 83, 8, 83, 6, 309]


## Spacy Tokenizer

In [0]:
eng_vocab_data = open("data/eng.vocab", "r")
msa_vocab_data = open("data/msa.vocab", "r")
lav_vocab_data = open("data/lav.vocab", "r")
mag_vocab_data = open("data/mag.vocab", "r")

eng_vocab_list = []
msa_vocab_list = []
lav_vocab_list = []
mag_vocab_list = []

for line in eng_vocab_data.readlines():
    eng_vocab_list.append(line.split("\t")[0])

for line in msa_vocab_data.readlines():
    msa_vocab_list.append(line.split("\t")[0])

for line in lav_vocab_data.readlines():
    lav_vocab_list.append(line.split("\t")[0])

for line in mag_vocab_data.readlines():
    mag_vocab_list.append(line.split("\t")[0])


In [0]:
# msa_vocab_list[0:200]

In [0]:
en_vocab = Vocab(strings=eng_vocab_list)
spacy_en_tokenizer = Tokenizer(en_vocab)

msa_vocab = Vocab(strings=msa_vocab_list)
spacy_msa_tokenizer = Tokenizer(msa_vocab)

lav_vocab = Vocab(strings=lav_vocab_list)
spacy_lav_tokenizer = Tokenizer(lav_vocab)

mag_vocab = Vocab(strings=mag_vocab_list)
spacy_mag_tokenizer = Tokenizer(mag_vocab)

## TF Tokenizer

https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer

In [57]:
ls data/model

eng.model  lav.model  mag.model  msa.model


In [0]:
tokenizer = T5Tokenizer('data/model/eng.model')
tokenizer.batch_encode_plus(data_English_MSA_both)
#msa_tokenizer = T5Tokenizer("data/msa.model")
#en_tokenizer = T5Tokenizer("data/eng.model")

In [0]:
input_ids = msa_tokenizer.encode('هل بامكاني الدخول؟ </s>', return_tensors='pt')

In [35]:
input_ids

tensor([[  82,    3,  170,  242,    6,   56, 1623,   19,    3,    2]])

In [36]:
#decode to make sure you can go back and forth between the encoding properly 
#@raef can you verify the Arabic?
output = msa_tokenizer.decode(input_ids[0])
print(output)

هل بامكاني الدخول؟ 


## Pytorch Data Set and Data Loader

https://github.com/google-research/text-to-text-transfer-transformer

pytorch dataset: https://pytorch.org/text/_modules/torchtext/datasets/translation.html

pytorch dataset documentation: https://torchtext.readthedocs.io/en/latest/datasets.html#iwslt

example dataset: https://iwslt2010.fbk.eu/node/32/

Field API: https://pytorch.org/text/data.html#torchtext.data.Field
Field Source: https://pytorch.org/text/_modules/torchtext/data/field.html#Field

https://pytorch.org/text/_modules/torchtext/data/utils.html#get_tokenizer
    
we need to pass our tokenizer as a function     

In [0]:
SRC = Field(tokenize = spacy_en_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = Field(tokenize = spacy_msa_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

# SRC = Field(tokenize = spacy_lav_tokenizer,
#             init_token = '<sos>',
#             eos_token = '<eos>',
#             lower = False)

# SRC = Field(tokenize = spacy_mag_tokenizer,
#             init_token = '<sos>',
#             eos_token = '<eos>',
#             lower = False)

In [0]:
eng_msa_train_dataset = TranslationDataset(path='data/train/train_eng_msa.', exts=('eng', 'msa'), fields=(SRC, TRG))
#lav_msa_train_dataset = TranslationDataset(path='data/train/train_lav_msa.', exts=('lav', 'msa'), fields=(SRC, TRG))
#mag_msa_train_dataset = TranslationDataset(path='data/train/train_mag_msa.', exts=('mag', 'msa'), fields=(SRC, TRG))

In [0]:
SRC.build_vocab(eng_msa_train_dataset)
TRG.build_vocab(eng_msa_train_dataset)
#SRC.build_vocab(lav_msa_train_dataset)
#TRG.build_vocab(lav_msa_train_dataset)
#SRC.build_vocab(mag_msa_train_dataset)
#TRG.build_vocab(mag_msa_train_dataset)

In [0]:
ex = eng_msa_train_dataset[0]

In [41]:
print(ex.src) 
print(ex.trg)

TRAIN_000\01\It was very kind of you to lend me an umbrella
TRAIN_000\01\كان من عظيم لطفك ان اعرتني مظلة


In [42]:
ls data/train

train_eng_msa.eng  train_lav_msa.lav  train_mag_msa.mag
train_eng_msa.msa  train_lav_msa.msa  train_mag_msa.msa


In [0]:
#train, validation and test are probably wrong.  
#https://github.com/pytorch/text/blob/master/torchtext/datasets/translation.py

'''
Arguments:
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            path (str): Common prefix of the splits' file paths, or None to use
                the result of cls.download(root).
            root: Root dataset storage directory. Default is '.data'.
            train: The prefix of the train data. Default: 'train'.
            validation: The prefix of the validation data. Default: 'val'.
            test: The prefix of the test data. Default: 'test'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.

'''


train_data, valid_data, test_data = eng_msa_train_dataset.splits(path= 'data/', train='train/train_eng_msa', validation='val/val_eng_msa', test='test/test_eng_msa', exts=('.eng', '.msa'),
                                                    fields = (SRC, TRG))

In [0]:
BATCH_SIZE =32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

## Model Training

https://huggingface.co/transformers/model_doc/t5.html#training

## Model Inference

https://github.com/huggingface/transformers/blob/master/examples/translation/t5/evaluate_wmt.py

In [0]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [0]:
from tqdm import tqdm
def generate_translations(lns, output_file_path, model_size, batch_size, device):
    model = T5ForConditionalGeneration.from_pretrained(model_size)
    model.to(device)
    #returned to using T5 tokenizer
    tokenizer = en_tokenizer

    task_specific_params = model.config.task_specific_params

    with Path(output_file_path).open("w") as output_file:
        for batch in tqdm(list(chunks(lns, batch_size))):
            batch = [text for text in batch]
            
            dct = tokenizer.batch_encode_plus(batch, max_length=512, return_tensors="pt", pad_to_max_length=True)

            input_ids = dct["input_ids"].to(device)
            attention_mask = dct["attention_mask"].to(device)

            translations = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            dec = [
                tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in translations
            ]

            for hypothesis in dec:
                output_file.write(hypothesis + "\n")

In [0]:
from pathlib import Path
def calculate_bleu_score(output_lns, refs_lns, score_path):
    bleu = corpus_bleu(output_lns, [refs_lns])
    result = "BLEU score: {}".format(bleu.score)
    with Path(score_path).open("w") as score_file:
        score_file.write(result)

In [0]:
!mkdir data/output/
!touch data/output/translated.msa
!touch data/output/score_eng_msa.txt

In [0]:
#got rid of the argparse and just put it in as params
def run_generate(model_size='t5-base', input_path='data/train/train_eng_msa.eng', output_path='data/output/translated.msa', ref_path='data/train/train_eng_msa.eng', score_path='data/output/score_eng_msa.txt', batch_size=BATCH_SIZE):
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")

    # Read input lines into python
    with open(input_path, "r") as input_file:
        input_lns = [x.strip() for x in input_file.readlines()]

    generate_translations(input_lns, output_path, model_size, batch_size, device)

    # Read generated lines into python
    with open(output_path, "r") as output_file:
        output_lns = [x.strip() for x in output_file.readlines()]

    # Read reference lines into python
    with open(ref_path, "r") as reference_file:
        refs_lns = [x.strip() for x in reference_file.readlines()]

    calculate_bleu_score(output_lns, refs_lns, args.score_path)

In [54]:
run_generate()

 44%|████▍     | 100/225 [1:52:17<2:20:22, 67.38s/it]

KeyboardInterrupt: ignored